# 前処理の手順と威力


In [1]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

warnings.filterwarnings('ignore')

%matplotlib inline

# データの準備

In [2]:
from sklearn.datasets import load_breast_cancer

cancer = load_breast_cancer()

X = cancer.data
y = cancer.target

print(X.shape)

(569, 30)


In [3]:
from sklearn.model_selection import train_test_split

X_trainval, X_test, y_trainval, y_test = train_test_split(X,
                                                          y,
                                                          train_size=0.80,
                                                          random_state=42)

In [4]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5, random_state=42)

# 予測モデルの構築(前処理なし)

In [5]:
from sklearn.svm import SVC
from sklearn.model_selection import ParameterGrid

svm_param_grid = {
    'C': [10**i for i in range(-3, 3)],
    'gamma': [10**i for i in range(-3, 3)]
}

svm_param_grid

{'C': [0.001, 0.01, 0.1, 1, 10, 100], 'gamma': [0.001, 0.01, 0.1, 1, 10, 100]}

In [6]:
# ParameterGridの使い方
for params in ParameterGrid(svm_param_grid):
    print(params)

{'C': 0.001, 'gamma': 0.001}
{'C': 0.001, 'gamma': 0.01}
{'C': 0.001, 'gamma': 0.1}
{'C': 0.001, 'gamma': 1}
{'C': 0.001, 'gamma': 10}
{'C': 0.001, 'gamma': 100}
{'C': 0.01, 'gamma': 0.001}
{'C': 0.01, 'gamma': 0.01}
{'C': 0.01, 'gamma': 0.1}
{'C': 0.01, 'gamma': 1}
{'C': 0.01, 'gamma': 10}
{'C': 0.01, 'gamma': 100}
{'C': 0.1, 'gamma': 0.001}
{'C': 0.1, 'gamma': 0.01}
{'C': 0.1, 'gamma': 0.1}
{'C': 0.1, 'gamma': 1}
{'C': 0.1, 'gamma': 10}
{'C': 0.1, 'gamma': 100}
{'C': 1, 'gamma': 0.001}
{'C': 1, 'gamma': 0.01}
{'C': 1, 'gamma': 0.1}
{'C': 1, 'gamma': 1}
{'C': 1, 'gamma': 10}
{'C': 1, 'gamma': 100}
{'C': 10, 'gamma': 0.001}
{'C': 10, 'gamma': 0.01}
{'C': 10, 'gamma': 0.1}
{'C': 10, 'gamma': 1}
{'C': 10, 'gamma': 10}
{'C': 10, 'gamma': 100}
{'C': 100, 'gamma': 0.001}
{'C': 100, 'gamma': 0.01}
{'C': 100, 'gamma': 0.1}
{'C': 100, 'gamma': 1}
{'C': 100, 'gamma': 10}
{'C': 100, 'gamma': 100}


In [7]:
best_acc = 0
best_params = dict()

for params in ParameterGrid(svm_param_grid):
    valid_scores = []

    for train_index, valid_index in skf.split(X_trainval, y_trainval):
        X_train = X_trainval[train_index]
        X_valid = X_trainval[valid_index]

        y_train = y_trainval[train_index]
        y_valid = y_trainval[valid_index]

        # clf: Classifier(分類器)の略語。よく使う省略形
        clf = SVC(**params) # `**` を使うといい感じに引数を渡せる
        # clf = SVC(C=params['C'], gamma=params['gamma']) と 同値
        
        # 訓練データでの学習
        clf.fit(X_train, y_train)

        # 検証データでの正答率を計算
        valid_score = clf.score(X_valid, y_valid)
        valid_scores.append(valid_score)
    
    mean_acc = np.mean(valid_scores)
    
    if mean_acc > best_acc:
        best_acc = mean_acc
        best_params = params

print(f'Best Acc: {best_acc:.4f}, BestParams: {best_params}')        

Best Acc: 0.9120, BestParams: {'C': 1, 'gamma': 0.001}


In [8]:
# 最適なパラメータ かつ trainval すべてを使って学習する
retrained_model = SVC(**best_params)
retrained_model.fit(X_trainval, y_trainval)

print(f'TrainVal Acc: {retrained_model.score(X_trainval, y_trainval)}')
print(f'Test Acc: {retrained_model.score(X_test, y_test)}') 

TrainVal Acc: 0.978021978021978
Test Acc: 0.9298245614035088


# 前処理の手順
- Scaling: データの値の範囲を何らかの方法で揃える
    - `MinMaxScaler()`
    - `StandardScaler()`

In [17]:
from sklearn.preprocessing import MinMaxScaler

dummy_data = np.array([[10], 
                       [8], 
                       [4],
                       [3], 
                       [5]])


min_max_scaler = MinMaxScaler()

min_max_scaler.fit(dummy_data)
min_max_scaler.transform(dummy_data)

array([[1.        ],
       [0.71428571],
       [0.14285714],
       [0.        ],
       [0.28571429]])

In [21]:
from sklearn.preprocessing import StandardScaler

# StandardScalerつかってDummyDataを変換してね
standard_scaler = StandardScaler()

standard_scaler.fit(dummy_data)
standard_scaler.transform(dummy_data)

array([[ 1.53392998],
       [ 0.76696499],
       [-0.76696499],
       [-1.15044748],
       [-0.38348249]])

# 前処理ありの交差検証

In [31]:
best_acc = 0
best_params = dict()

for params in ParameterGrid(svm_param_grid):
    valid_scores = []

    for train_index, valid_index in skf.split(X_trainval, y_trainval):
        X_train = X_trainval[train_index]
        X_valid = X_trainval[valid_index]

        y_train = y_trainval[train_index]
        y_valid = y_trainval[valid_index]

        clf = SVC(**params)
        
        # 標準化
        scaler = StandardScaler()
        scaler.fit(X_train)  # 訓練データのみに fit() させる！
        
        X_train_scaled = scaler.transform(X_train)
        X_valid_scaled = scaler.transform(X_valid)
        
        # スケーリング済みの訓練データでの学習
        clf.fit(X_train_scaled, y_train)

        valid_score = clf.score(X_valid_scaled, y_valid)
        valid_scores.append(valid_score)
    
    mean_acc = np.mean(valid_scores)
    
    if mean_acc > best_acc:
        best_acc = mean_acc
        best_params = params

print(f'Best Acc: {best_acc:.4f}, BestParams: {best_params}')        

Best Acc: 0.9757, BestParams: {'C': 10, 'gamma': 0.01}


おわり